In [1]:
import pandas as pd
import json
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
import tqdm
import os


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Data processing
## Loading all data

In [3]:
BASE_PATH = '/content/drive/MyDrive/AI/Football Betting/'
DATA_PATH = BASE_PATH + 'new_data.json'

with open(DATA_PATH, 'r') as f:
    data = json.load(f)

len(data)

6841

## Selecting teams

In [4]:
teams = []
for idx, item in enumerate(data):
    if item["match"]["HomeTeam"] == 0 or item["match"]["AwayTeam"] == 0:
        del data[idx]
        continue
    if item["match"]["HomeTeam"] not in teams:
        teams.append(item["match"]["HomeTeam"])
    if item["match"]["AwayTeam"] not in teams:
        teams.append(item["match"]["AwayTeam"])

In [5]:
len(data), len(teams)

(6840, 42)

## Teams and results tokenization

In [6]:
team2id = {team: id_+1 for id_, team in enumerate(sorted(teams))}

In [7]:
result2id = {
    "H": 0,
    "D": 1,
    "A": 2,
}

## Changing data into tensors

In [8]:
def match2tensor(item):
    a_team = [team2id[item["HomeTeam"]]]+ list(item["HomeTeamData"].values())[:-1] # without position
    b_team = [team2id[item["AwayTeam"]]] + list(item["AwayTeamData"].values())[:-1] # without position
    result = result2id[item["FTR"]]
    match_tensor = torch.tensor([a_team, b_team], dtype=torch.float32)
    return match_tensor, result

def matches2tensor(item):
    matches_tensor = None
    for key in list(item.keys())[1:]:
        if key in ["HomeTeam", "AwayTeam"]:
            tensor =  torch.tensor(
                [team2id[i] if isinstance(i, str) else i for i in item[key]]
            )
        elif  key in ["FTR", "HTR"]:
            tensor = torch.tensor(
                [result2id[i] if isinstance(i, str) else i for i in item[key]]
            )
        else:
            tensor = torch.tensor(item[key])
        tensor = torch.unsqueeze(tensor, dim=-1)
        if matches_tensor is None:
            matches_tensor = tensor
        else:
            matches_tensor = torch.cat([matches_tensor, tensor], dim=-1)
    return matches_tensor

def table2tensor(item):
    table = []

    for key in item.keys():
        if key == '0':
            continue
        row = [team2id[key]] + list(item[key].values())
        table.append(row)

    table_tensor = torch.tensor(table, dtype=torch.float32)
    return table_tensor

In [9]:
matches = []
tables = []
matches_ab = []
matches_a = []
matches_b = []
bets = []
labels = []

for item in data:
    match_tensor, result = match2tensor(item["match"])
    matches_ab_tensor = matches2tensor(item["matches_ab"])
    matches_a_tensor = matches2tensor(item["matches_a"])
    matches_b_tensor = matches2tensor(item["matches_b"])
    table_tensor = table2tensor(item["table"])
    bet_tensor = torch.tensor(item["bets"])
    matches.append(match_tensor.type(torch.float32))
    labels.append(result)
    matches_ab.append(matches_ab_tensor.type(torch.float32))
    matches_a.append(matches_a_tensor.type(torch.float32))
    matches_b.append(matches_b_tensor.type(torch.float32))
    tables.append(table_tensor.type(torch.float32))
    bets.append(bet_tensor.type(torch.float32))

matches_tensor = torch.stack(matches).type(torch.float32)
tables_tensor = torch.stack(tables).type(torch.float32)
matches_ab_tensor = torch.stack(matches_ab).type(torch.float32)
matches_a_tensor = torch.stack(matches_a).type(torch.float32)
matches_b_tensor = torch.stack(matches_b).type(torch.float32)
bets_tensor = torch.stack(bets).type(torch.float32)
labels_tensor = torch.tensor(labels)
y = torch.tensor(labels)

In [10]:
matches_tensor.shape, tables_tensor.shape, matches_ab_tensor.shape, matches_a_tensor.shape, matches_b_tensor.shape, bets_tensor.shape

(torch.Size([6840, 2, 8]),
 torch.Size([6840, 20, 8]),
 torch.Size([6840, 5, 20]),
 torch.Size([6840, 5, 20]),
 torch.Size([6840, 5, 20]),
 torch.Size([6840, 6]))

## Scaling data

In [11]:
from sklearn.preprocessing import MinMaxScaler

table_scaler = MinMaxScaler()
stats_scaler = MinMaxScaler()
bets_scaler = MinMaxScaler()

table_scaler.fit(
    torch.cat([
        matches_tensor.reshape(len(matches_tensor)*matches_tensor.shape[1], matches_tensor.shape[2])[:, 1:],
        tables_tensor.reshape(len(tables_tensor)*tables_tensor.shape[1], tables_tensor.shape[2])[:, 1:],
    ], dim=-2)
)
stats_scaler.fit(
    torch.cat([
        matches_ab_tensor.reshape(len(matches_ab_tensor)*matches_ab_tensor.shape[1], matches_ab_tensor.shape[2])[:, 2:],
        matches_a_tensor.reshape(len(matches_a_tensor)*matches_a_tensor.shape[1], matches_a_tensor.shape[2])[:, 2:],
        matches_b_tensor.reshape(len(matches_b_tensor)*matches_b_tensor.shape[1], matches_b_tensor.shape[2])[:, 2:],
    ], dim=-2)
)
bets_scaler.fit(bets_tensor[:, :3])

MinMaxScaler()

In [12]:
stats_shape = (matches_tensor.shape[0], matches_tensor.shape[1], matches_tensor.shape[2] - 1)
scale_shape = (len(matches_tensor)*matches_tensor.shape[1], matches_tensor.shape[2]-1)
matches_tensor[:, :, 1:] = torch.from_numpy(table_scaler.transform(matches_tensor[:, :, 1:].reshape(scale_shape))).reshape(stats_shape)

stats_shape = (tables_tensor.shape[0], tables_tensor.shape[1], tables_tensor.shape[2] - 1)
scale_shape = (len(tables_tensor)*tables_tensor.shape[1], tables_tensor.shape[2]-1)
tables_tensor[:, :, 1:] = torch.from_numpy(table_scaler.transform(tables_tensor[:, :, 1:].reshape(scale_shape))).reshape(stats_shape)

stats_shape = (matches_ab_tensor.shape[0], matches_ab_tensor.shape[1], matches_ab_tensor.shape[2] - 2)
scale_shape = (len(matches_ab_tensor)*matches_ab_tensor.shape[1], matches_ab_tensor.shape[2]-2)
matches_ab_tensor[:, :, 2:] = torch.from_numpy(stats_scaler.transform(matches_ab_tensor[:, :, 2:].reshape(scale_shape))).reshape(stats_shape)
matches_a_tensor[:, :, 2:] = torch.from_numpy(stats_scaler.transform(matches_a_tensor[:, :, 2:].reshape(scale_shape))).reshape(stats_shape)
matches_b_tensor[:, :, 2:] = torch.from_numpy(stats_scaler.transform(matches_b_tensor[:, :, 2:].reshape(scale_shape))).reshape(stats_shape)

bets_tensor = bets_tensor[:, :3]
bets_tensor = bets_scaler.transform(bets_tensor)

## Spliting data into train, valid nad test sets

In [13]:
X = []

for tensors in zip(matches_tensor, matches_ab_tensor, matches_a_tensor, matches_b_tensor, tables_tensor, bets_tensor):
    X.append(tensors)

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [15]:
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5)

## Creating datasets and dataloaders

In [16]:
class FootballBettingDataset(Dataset):
    def __init__(self, X: list[torch.Tensor], y: list[torch.Tensor]) -> None:
        super(FootballBettingDataset, self).__init__()
        self.X = X
        self.y = y

    def __len__(self) -> int:
        return len(self.y)

    def __getitem__(self, index: int) -> tuple[list, int]:
        return self.X[index], self.y[index]

In [17]:
BATCH_SIZE = 32

train_dataloader = DataLoader(
    FootballBettingDataset(X_train, y_train),
    batch_size=BATCH_SIZE,
    shuffle=True,
)

valid_dataloader = DataLoader(
    FootballBettingDataset(X_valid, y_valid),
    batch_size=BATCH_SIZE,
    shuffle=False,
)

test_dataloader = DataLoader(
    FootballBettingDataset(X_test, y_test),
    batch_size=BATCH_SIZE,
    shuffle=False,
)

# Football better

In [18]:
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)

  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_name = f"{model.d_model}_{model_name}"
  model_save_path = target_dir_path / model_name

  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

In [19]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: str):

    model.train()

    train_loss, train_acc = 0, 0

    for batch, (X, y) in enumerate(dataloader):
        X, y = X, y.to(device)

        match, matches_ab, matches_a, matches_b, _, _ = X
        match = match.to(device)
        matches_ab = matches_ab.to(device)
        matches_a = matches_a.to(device)
        matches_b = matches_b.to(device)

        y_pred = model(match, matches_ab, matches_a, matches_b)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: str):
    model.eval()

    test_loss, test_acc = 0, 0

    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X, y.to(device)

            match, matches_ab, matches_a, matches_b, _, _ = X
            match = match.to(device)
            matches_ab = matches_ab.to(device)
            matches_a = matches_a.to(device)
            matches_b = matches_b.to(device)

            test_preds = model(match, matches_ab, matches_a, matches_b)
            loss = loss_fn(test_preds, y)
            test_loss += loss.item()

            test_pred_labels = test_preds.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

from tqdm.auto import tqdm

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          device: str,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5):

    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }

    model.to(device)

    if os.path.exists(BASE_PATH+"best_results.json"):
        with open(BASE_PATH+"best_results.json", 'r') as f:
            best_results = json.load(f)
    else:
        best_results = {"test_loss": 5, "test_acc": 0}

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,
                                           device=device)
        test_loss, test_acc = test_step(model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn,
            device=device)

        if test_acc > best_results["test_acc"]:
            best_results["test_acc"] = test_acc
            save_model(model, BASE_PATH+'best_acc_model', 'model.pt')

        if test_loss < best_results["test_loss"]:
            best_results["test_loss"] = test_loss
            save_model(model, BASE_PATH+'best_loss_model', 'model.pt')

        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    with open(BASE_PATH+'best_results.json', 'w+') as f:
        json.dump(best_results, f)

    return results

In [20]:
matches_tensor.shape, tables_tensor.shape, matches_ab_tensor.shape, matches_a_tensor.shape, matches_b_tensor.shape

(torch.Size([6840, 2, 8]),
 torch.Size([6840, 20, 8]),
 torch.Size([6840, 5, 20]),
 torch.Size([6840, 5, 20]),
 torch.Size([6840, 5, 20]))

## Building the model

In [74]:
class PastAnalyser(nn.Module):
    def __init__(self, embedding_layer, d_model) -> None:
        super().__init__()
        self.embedding = embedding_layer
        self.lstm = nn.LSTM(18, d_model, 3, batch_first=True)
        self.cnn = nn.Conv1d(5, 5, 3, padding=1)

        self.fc_block1 = nn.Sequential(
            nn.Linear(2*d_model, 3*d_model),
            nn.GELU(),
            nn.Linear(3*d_model, d_model),
            nn.GELU()
        )

        self.fc_block2 = nn.Sequential(
            nn.Linear(5*d_model, 5*d_model),
            nn.GELU(),
            nn.Linear(5*d_model, d_model),
            nn.GELU(),
        )

    def forward(self, x):
        embedded_teams = self.embedding(x[:, :, :2].type(torch.int))
        flattened_embedded_teams = embedded_teams.flatten(-2)
        strength_comparisson = self.fc_block1(flattened_embedded_teams)
        x = self.cnn(x[:, :, 2:].type(torch.float32))
        x, _ = self.lstm(x)
        x = torch.add(strength_comparisson, x)
        x = torch.flatten(x, -2, -1)
        x = self.fc_block2(x)
        return x

In [112]:
class Model(nn.Module):
    def __init__(self, d_model) -> None:
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(len(teams)+1, d_model)
        self.past_analyser = PastAnalyser(self.embedding, d_model)
        self.fc_block1 = nn.Sequential(
            nn.Linear(7, 2*d_model),
            nn.ReLU(),
            nn.Linear(2*d_model, d_model),
            nn.ReLU(),
        )

        self.fc_block2 = nn.Sequential(
            nn.Linear(2*d_model, 4*d_model),
            nn.ReLU(),
            nn.Linear(4*d_model, d_model),
            nn.ReLU(),
        )

        self.fc_block3 = nn.Sequential(
            nn.Linear(4*d_model, 8*d_model),
            nn.ReLU(),
            nn.Linear(8*d_model, 3),
            nn.Softmax(-1),
        )

    def forward(self, match, matches_ab, matches_a, matches_b):
        embedded_teams = self.embedding(match[:, :, :1].type(torch.int))
        embedded_teams = torch.flatten(embedded_teams, -2)
        x = self.fc_block1(match[:, :, 1:].type(torch.float32))
        x = x + embedded_teams
        x = torch.flatten(x, -2)
        x = self.fc_block2(x)

        matches_ab_vector = self.past_analyser(matches_ab)
        matches_a_vector = self.past_analyser(matches_a)
        matches_b_vector = self.past_analyser(matches_b)

        x = torch.concat([x, matches_ab_vector, matches_a_vector, matches_b_vector], dim=-1)

        x = self.fc_block3(x)

        return x

model = Model(3)
match, matches_ab, matches_a, matches_b = matches_tensor[:1, :, :], \
                                          matches_ab_tensor[:1, :, :], \
                                          matches_a_tensor[:1, :, :], \
                                          matches_b_tensor[:1, :, :]
model(match, matches_ab, matches_a, matches_b).shape

torch.Size([1, 3])

In [160]:
class KQVSelfAttention(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.kqv = nn.ModuleList([nn.Linear(d_model, d_model) for _ in range(3)])

    def forward(self, x):
        k, q, v = [l(x) for l in self.kqv]
        attention_scores = torch.matmul(q, k.transpose(-2, -1))
        attention_weights = torch.nn.functional.softmax(attention_scores, dim=-1)
        x = torch.matmul(attention_weights, v)
        return x

In [161]:
KQVSelfAttention(4)(torch.randn(1, 2, 4))

tensor([[[ 0.6169,  0.6313,  0.1903, -0.2100],
         [ 0.6160,  0.6325,  0.1886, -0.2087]]], grad_fn=<UnsafeViewBackward0>)

In [162]:
class TeamsComparissonModel(nn.Module):
    def __init__(self, embedding_layer, d_model):
        super().__init__()
        self.embedding = embedding_layer
        self.kqv_self_attention = KQVSelfAttention(d_model)

    def forward(self, x):
        x = self.embedding(x)
        x = self.kqv_self_attention(x)
        x = torch.sum(x, dim=-2)/2
        return x

In [180]:
class PastAnalyser(nn.Module):
    def __init__(self, embedding_layer, d_model) -> None:
        super().__init__()
        self.teams_comparisson_model = TeamsComparissonModel(embedding_layer, d_model)
        self.lstm1 = nn.LSTM(18, d_model, 3, batch_first=True)
        self.lstm2 = nn.LSTM(2*d_model, d_model, 3, batch_first=True)
        self.cnn = nn.Conv1d(5, 5, 3, padding=1)

        self.fc_block1 = nn.Sequential(
            nn.Linear(2*d_model, 4*d_model),
            nn.GELU(),
            nn.Linear(4*d_model, 2*d_model),
            nn.GELU()
        )

    def forward(self, x):
        x1 = self.teams_comparisson_model(x[:, :, :2].type(torch.int))

        x2 = self.cnn(x[:, :, 2:].type(torch.float32))
        x2, _ = self.lstm1(x2)
        x = torch.concat([x1, x2], -1)
        x = self.fc_block1(x)
        x, _ = self.lstm2(x)
        return x[:, -1, :]

In [181]:
x = torch.tensor([[[1, 2], [1, 2], [1, 2], [1, 2], [1, 2]]])
x = torch.concat([x, torch.randn(1, 5, 16)], -1)
print(x.shape)
d_model = 8
model = PastAnalyser(nn.Embedding(len(teams)+1, d_model), d_model)
model(x)

torch.Size([1, 5, 18])


tensor([[ 0.0556,  0.0844,  0.1986, -0.0349,  0.1083, -0.0554, -0.1635, -0.0425]],
       grad_fn=<SliceBackward0>)

In [182]:
class Model(nn.Module):
    def __init__(self, d_model) -> None:
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(len(teams)+1, d_model)
        self.past_analyser = PastAnalyser(self.embedding, d_model)
        self.teams_comparisson_model = TeamsComparissonModel(self.embedding, d_model)
        self.stats_comparisson = KQVSelfAttention(7)

        self.fc_block1 = nn.Sequential(
            nn.Linear(7*2, 2*d_model),
            nn.ReLU(),
            nn.Linear(2*d_model, d_model),
            nn.ReLU(),
        )

        self.fc_block2 = nn.Sequential(
            nn.Linear(2*d_model, 4*d_model),
            nn.ReLU(),
            nn.Linear(4*d_model, d_model),
            nn.ReLU(),
        )

        self.fc_block3 = nn.Sequential(
            nn.Linear(4*d_model, 8*d_model),
            nn.ReLU(),
            nn.Linear(8*d_model, 3),
            nn.Softmax(-1),
        )

    def forward(self, match, matches_ab, matches_a, matches_b):
        x1 = self.teams_comparisson_model(torch.squeeze(match[:, :, :1], -1).type(torch.int))

        x2 = self.stats_comparisson(match[:, :, 1:].type(torch.float32))
        x2 = self.fc_block1(torch.flatten(x2, -2))

        x = torch.concat([x1, x2], -1)
        x = self.fc_block2(x)

        matches_ab_vector = self.past_analyser(matches_ab)
        matches_a_vector = self.past_analyser(matches_a)
        matches_b_vector = self.past_analyser(matches_b)

        x = torch.concat([x, matches_ab_vector, matches_a_vector, matches_b_vector], dim=-1)
        x = self.fc_block3(x)
        return x

model = Model(3)
match, matches_ab, matches_a, matches_b = matches_tensor[:1, :, :], \
                                          matches_ab_tensor[:1, :, :], \
                                          matches_a_tensor[:1, :, :], \
                                          matches_b_tensor[:1, :, :]
model(match, matches_ab, matches_a, matches_b).shape

torch.Size([1, 3])

## Running the training loop

In [188]:
d_model = 32
torch.manual_seed(42)
model = Model(d_model)

# model.load_state_dict(torch.load(BASE_PATH + "best_acc_model/32_model.pt"))

In [189]:
NUM_EPOCHS = 180

device = "cuda" if torch.cuda.is_available() else 'cpu'

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)

model_0_results = train(model=model,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        device=device,
                        optimizer=optimizer,
                        loss_fn=loss_fn,
                        epochs=NUM_EPOCHS)


  0%|          | 0/180 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0832 | train_acc: 0.4301 | test_loss: 1.0614 | test_acc: 0.4669
Epoch: 2 | train_loss: 1.0607 | train_acc: 0.4626 | test_loss: 1.0595 | test_acc: 0.4669
Epoch: 3 | train_loss: 1.0591 | train_acc: 0.4625 | test_loss: 1.0587 | test_acc: 0.4669
Epoch: 4 | train_loss: 1.0556 | train_acc: 0.4623 | test_loss: 1.0559 | test_acc: 0.4669
Epoch: 5 | train_loss: 1.0401 | train_acc: 0.4745 | test_loss: 1.0382 | test_acc: 0.4858
Epoch: 6 | train_loss: 1.0164 | train_acc: 0.5128 | test_loss: 1.0268 | test_acc: 0.5009
Epoch: 7 | train_loss: 1.0072 | train_acc: 0.5183 | test_loss: 1.0156 | test_acc: 0.5189
Epoch: 8 | train_loss: 1.0009 | train_acc: 0.5232 | test_loss: 1.0120 | test_acc: 0.5180
Epoch: 9 | train_loss: 0.9979 | train_acc: 0.5295 | test_loss: 1.0106 | test_acc: 0.5227
Epoch: 10 | train_loss: 0.9962 | train_acc: 0.5307 | test_loss: 1.0107 | test_acc: 0.5256
Epoch: 11 | train_loss: 0.9942 | train_acc: 0.5346 | test_loss: 1.0125 | test_acc: 0.5246
Epoch: 12 | train_l

KeyboardInterrupt: ignored

# Decision maker using Binary Crossentropy loss

In [ ]:
class DecisionMaker(nn.Module):
    def __init__(self, d_model, hidden_units) -> None:
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(len(teams)+1, d_model)
        self.fc_block1 = nn.Sequential(
            nn.Linear(2*d_model, hidden_units),
            nn.ReLU()
        )
        self.fc_block2 = nn.Sequential(
            nn.Linear(3, hidden_units),
            nn.ReLU(),
        )
        self.fc_block3 = nn.Sequential(
            nn.Linear(3, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, hidden_units),
            nn.ReLU(),
        )
        self.fc_block4 = nn.Sequential(
            nn.Linear(3*hidden_units, 4*hidden_units),
            nn.ReLU(),
            nn.Linear(4*hidden_units, 1),
            nn.Sigmoid()
        )
        self.fc_block5 = nn.Sequential(
            nn.Linear(3*hidden_units, 4*hidden_units),
            nn.ReLU(),
            nn.Linear(4*hidden_units, 1),
            nn.Sigmoid()
        )

    def forward(self, teams, probabilities, bets):
        embedded_teams = self.embedding(teams.type(torch.int))
        embedded_teams = torch.flatten(embedded_teams, -3)
        x1 = self.fc_block1(embedded_teams.type(torch.float32))
        x2 = self.fc_block2(probabilities)
        x3 = self.fc_block3(bets)
        x = torch.concat([x1, x2, x3], dim=-1)
        y1 = self.fc_block4(x)
        return y1

In [ ]:
model = Model(32)
model.load_state_dict(torch.load(BASE_PATH + "best_loss_model/32_model.pt"))

decision_maker = DecisionMaker(32, 32)
decision_maker(torch.tensor([[3], [4]]), torch.tensor([0.3, 0.4, 0.3]), torch.tensor([1., 2., 3.]))

tensor([0.4948], grad_fn=<SigmoidBackward0>)

In [ ]:
model.train()
device = "cuda" if torch.cuda.is_available() else 'cpu'

decision_maker.to(device)
model.to(device)

loss_fn = nn.BCELoss()
optimizer = torch.optim.AdamW(decision_maker.parameters(), lr=1e-2)

EPOCHS = 100

for epoch in range(EPOCHS):
    starting_balance = 5
    balance = starting_balance

    train_loss, train_return = 0, 0

    if os.path.exists(BASE_PATH + "decision_model_best_results.json"):
        with open(BASE_PATH + "decision_model_best_results.json", 'r') as f:
            best_results = json.load(f)
    else:
        best_results = {'test_return': 0}

    decision_maker.train()

    for X, y in train_dataloader:
        X, y = X, y.to(device)

        match, matches_ab, matches_a, matches_b, _, bets = X
        match, matches_ab, matches_a, matches_b, bets = match.to(device), matches_ab.to(device), matches_a.to(device), matches_b.to(device), bets.to(device)
        y_pred = model(match, matches_ab, matches_a, matches_b)
        original_bets = torch.from_numpy(bets_scaler.inverse_transform(bets.cpu())).to(device)


        pred_labels = y_pred.argmax(dim=1)

        match_between = match[:, :, :1].type(torch.int)
        how_much_to_invest = decision_maker(match_between, y_pred, bets.type(torch.float32))

        target = (pred_labels == y) == 1
        loss = loss_fn(how_much_to_invest, torch.unsqueeze(target.type(torch.float32), dim=-1))
        train_loss += loss
        returns = torch.where(target, torch.squeeze(how_much_to_invest)*(original_bets[range(len(original_bets)), pred_labels.tolist()]-1) + 1, 1-torch.squeeze(how_much_to_invest))
        train_return += torch.prod(returns)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss = train_loss / len(train_dataloader)
    train_return = train_return / len(train_dataloader)

    decision_maker.eval()
    test_loss, test_return, test_acc = 0, 0, 0

    with torch.inference_mode():
        for X, y in valid_dataloader:
            X, y = X, y.to(device)

            match, matches_ab, matches_a, matches_b, _, bets = X
            match, matches_ab, matches_a, matches_b, bets = match.to(device), matches_ab.to(device), matches_a.to(device), matches_b.to(device), bets.to(device)

            original_bets = torch.from_numpy(bets_scaler.inverse_transform(bets.cpu())).to(device)


            y_pred = model(match, matches_ab, matches_a, matches_b)
            pred_labels = y_pred.argmax(dim=1)


            match_between = match[:, :, :1].type(torch.int)

            how_much_to_invest = decision_maker(match_between, y_pred, bets.type(torch.float32))

            target = (pred_labels == y) == 1

            loss = loss_fn(how_much_to_invest, torch.unsqueeze(target.type(torch.float32), dim=-1))
            test_loss += loss
            returns = torch.where(target, torch.squeeze(how_much_to_invest)*(original_bets[range(len(original_bets)), pred_labels.tolist()]-1) + 1, 1-torch.squeeze(how_much_to_invest))
            test_return += torch.prod(returns)

            test_pred_labels = how_much_to_invest > 0.5
            test_acc += ((torch.squeeze(test_pred_labels) == target).sum().item()/len(test_pred_labels))

    test_loss = test_loss / len(valid_dataloader)
    test_return = test_return / len(valid_dataloader)
    test_acc = test_acc / len(valid_dataloader)

    if test_return > best_results['test_return']:
        best_results['test_return'] = test_return.item()
        save_model(decision_maker, BASE_PATH + 'decision_maker', 'model.pt')

        with open(BASE_PATH + 'decision_model_best_results.json', 'w+') as f:
            json.dump(best_results, f)

    if epoch%4 == 0:
        print(
                f"Epoch: {epoch+1} | "
                f"train_loss: {train_loss:.4f} | "
                f"train_return: {train_return:.4f} | "
                f"test_loss: {test_loss:.4f} | "
                f"test_return: {test_return:.4f} | "
                f"test_acc: {test_acc:.4f}"
            )


Epoch: 1 | train_loss: 0.6709 | train_return: 0.6722 | test_loss: 0.6619 | test_return: 0.1303 | test_acc: 0.6032
Epoch: 5 | train_loss: 0.6412 | train_return: 0.4342 | test_loss: 0.6670 | test_return: 0.1353 | test_acc: 0.5871
Epoch: 9 | train_loss: 0.6250 | train_return: 1.8239 | test_loss: 0.6791 | test_return: 0.1481 | test_acc: 0.5994
Epoch: 13 | train_loss: 0.6074 | train_return: 1.7224 | test_loss: 0.7038 | test_return: 0.1731 | test_acc: 0.6127
Epoch: 17 | train_loss: 0.6014 | train_return: 5.5225 | test_loss: 0.7178 | test_return: 0.1169 | test_acc: 0.6032
Epoch: 21 | train_loss: 0.5875 | train_return: 11.2564 | test_loss: 0.7199 | test_return: 0.1324 | test_acc: 0.5672
Epoch: 25 | train_loss: 0.5785 | train_return: 5.7119 | test_loss: 0.7519 | test_return: 0.2179 | test_acc: 0.5559
Epoch: 29 | train_loss: 0.5724 | train_return: 15.1783 | test_loss: 0.7880 | test_return: 0.1216 | test_acc: 0.5720
Epoch: 33 | train_loss: 0.5699 | train_return: 3.0433 | test_loss: 0.7984 | test_

In [ ]:
decision_maker.load_state_dict(torch.load(BASE_PATH + "decision_maker/32_model.pt"))

<All keys matched successfully>

In [ ]:
decision_maker.eval()
test_loss, test_return, test_acc = 0, 0, 0

with torch.inference_mode():
    for X, y in test_dataloader:
        X, y = X, y.to(device)

        match, matches_ab, matches_a, matches_b, _, bets = X
        match, matches_ab, matches_a, matches_b, bets = match.to(device), matches_ab.to(device), matches_a.to(device), matches_b.to(device), bets.to(device)

        original_bets = torch.from_numpy(bets_scaler.inverse_transform(bets.cpu())).to(device)


        y_pred = model(match, matches_ab, matches_a, matches_b)
        pred_labels = y_pred.argmax(dim=1)


        match_between = match[:, :, :1].type(torch.int)

        how_much_to_invest = decision_maker(match_between, y_pred, bets.type(torch.float32))

        target = (pred_labels == y) == 1

        loss = loss_fn(how_much_to_invest, torch.unsqueeze(target.type(torch.float32), dim=-1))
        test_loss += loss
        returns = torch.where(target, torch.squeeze(how_much_to_invest)*(original_bets[range(len(original_bets)), pred_labels.tolist()]-1) + 1, 1-torch.squeeze(how_much_to_invest))
        test_return += torch.prod(returns)

        test_pred_labels = how_much_to_invest > 0.5
        test_acc += ((torch.squeeze(test_pred_labels) == target).sum().item()/len(test_pred_labels))

test_loss = test_loss / len(test_dataloader)
test_return = test_return / len(test_dataloader)
test_acc = test_acc / len(test_dataloader)

test_loss.cpu().item(), test_return.cpu().item(), test_acc

(0.6010051369667053, 20.04970127339357, 0.7433712121212122)

#

# Decision maker with RL attitude

In [ ]:
class DecisionMaker(nn.Module):
    def __init__(self, d_model, hidden_units) -> None:
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(len(teams)+1, d_model)
        self.fc_block1 = nn.Sequential(
            nn.Linear(2*d_model, hidden_units),
            nn.ReLU()
        )
        self.fc_block2 = nn.Sequential(
            nn.Linear(3, hidden_units),
            nn.ReLU(),
        )
        self.fc_block3 = nn.Sequential(
            nn.Linear(3, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, hidden_units),
            nn.ReLU(),
        )
        self.fc_block4 = nn.Sequential(
            nn.Linear(3*hidden_units, 4*hidden_units),
            nn.ReLU(),
            nn.Linear(4*hidden_units, 1),
            nn.Sigmoid()
        )
        self.fc_block5 = nn.Sequential(
            nn.Linear(3*hidden_units, 4*hidden_units),
            nn.ReLU(),
            nn.Linear(4*hidden_units, 1),
            nn.Sigmoid()
        )

    def forward(self, teams, probabilities, bets):
        embedded_teams = self.embedding(teams.type(torch.int))
        embedded_teams = torch.flatten(embedded_teams, -3)
        x1 = self.fc_block1(embedded_teams.type(torch.float32))
        x2 = self.fc_block2(probabilities)
        x3 = self.fc_block3(bets)
        x = torch.concat([x1, x2, x3], dim=-1)
        to_invest = self.fc_block4(x)
        how_much_to_invest = self.fc_block5(x)
        return to_invest, how_much_to_invest

In [ ]:
model = Model(32)
model.load_state_dict(torch.load(BASE_PATH + "best_loss_model/32_model.pt"))

decision_maker = DecisionMaker(32, 32)
decision_maker(torch.tensor([[3], [4]]), torch.tensor([0.3, 0.4, 0.3]), torch.tensor([1., 2., 3.]))

(tensor([0.5202], grad_fn=<SigmoidBackward0>),
 tensor([0.4889], grad_fn=<SigmoidBackward0>))

In [ ]:
model.train()
device = "cuda" if torch.cuda.is_available() else 'cpu'

decision_maker.to(device)
model.to(device)

mae_loss = nn.L1Loss()
bce_loss = nn.BCELoss()

optimizer = torch.optim.AdamW(decision_maker.parameters(), lr=5e-3)

EPOCHS = 1

for epoch in range(EPOCHS):
    starting_balance = 5
    balance = starting_balance

    train_loss, train_return = 0, 0

    if os.path.exists(BASE_PATH + "decision_model_best_results_rl.json"):
        with open(BASE_PATH + "decision_model_best_results_rl.json", 'r') as f:
            best_results = json.load(f)
    else:
        best_results = {'test_return': 0}

    decision_maker.train()

    for X, y in train_dataloader:
        X, y = X, y.to(device)

        match, matches_ab, matches_a, matches_b, _, bets = X
        match, matches_ab, matches_a, matches_b, bets = match.to(device), matches_ab.to(device), matches_a.to(device), matches_b.to(device), bets.to(device)
        y_pred = model(match, matches_ab, matches_a, matches_b)
        original_bets = torch.from_numpy(bets_scaler.inverse_transform(bets.cpu())).to(device)

        pred_labels = y_pred.argmax(dim=1)
        match_between = match[:, :, :1].type(torch.int)
        to_invest, how_much_to_invest = decision_maker(match_between, y_pred, bets.type(torch.float32))

        to_invest = to_invest > 0.5

        rewards = torch.where(
            to_invest,
        )


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss = train_loss / len(train_dataloader)
    train_return = train_return / len(train_dataloader)

    decision_maker.eval()
    test_loss, test_return, test_acc = 0, 0, 0

    with torch.inference_mode():
        for X, y in test_dataloader:
            break
            X, y = X, y.to(device)

            match, matches_ab, matches_a, matches_b, _, bets = X
            match, matches_ab, matches_a, matches_b, bets = match.to(device), matches_ab.to(device), matches_a.to(device), matches_b.to(device), bets.to(device)

            original_bets = torch.from_numpy(bets_scaler.inverse_transform(bets.cpu())).to(device)


            y_pred = model(match, matches_ab, matches_a, matches_b)
            pred_labels = y_pred.argmax(dim=1)


            match_between = match[:, :, :1].type(torch.int)

            how_much_to_invest = decision_maker(match_between, y_pred, bets.type(torch.float32))

            target = (pred_labels == y) == 1

            loss = loss_fn(how_much_to_invest, torch.unsqueeze(target.type(torch.float32), dim=-1))
            test_loss += loss
            returns = torch.where(target, torch.squeeze(how_much_to_invest)*(original_bets[range(len(original_bets)), pred_labels.tolist()]-1) + 1, 1-torch.squeeze(how_much_to_invest))
            test_return += torch.prod(returns)

            test_pred_labels = how_much_to_invest > 0.5
            test_acc += ((torch.squeeze(test_pred_labels) == target).sum().item()/len(test_pred_labels))

    test_loss = test_loss / len(test_dataloader)
    test_return = test_return / len(test_dataloader)
    test_acc = test_acc / len(test_dataloader)

    if test_return > best_results['test_return']:
        best_results['test_return'] = test_return.item()
        save_model(decision_maker, BASE_PATH + 'decision_maker', 'model.pt')

        with open(BASE_PATH + 'decision_model_best_results_rl.json', 'w+') as f:
            json.dump(best_results, f)

    if epoch%4 == 0:
        print(
                f"Epoch: {epoch+1} | "
                f"train_loss: {train_loss:.4f} | "
                f"train_return: {train_return:.4f} | "
                f"test_loss: {test_loss:.4f} | "
                f"test_return: {test_return:.4f} | "
                f"test_acc: {test_acc:.4f}"
            )


# Saving test results into file

In [ ]:
best_acc_model = Model(32)
best_acc_model.load_state_dict(torch.load(BASE_PATH + "best_acc_model/32_model.pt"))
best_loss_model = Model(32)
best_loss_model.load_state_dict(torch.load(BASE_PATH + "best_loss_model/32_model.pt"))

columns = ["real", "pred", "certainty", "home certainty", "draw certainty", "away ceratinity", "home", "away", "how much to invest",  "B365H","B365D", "B365A"]
best_acc_logs = pd.DataFrame(columns=columns)
best_loss_logs = pd.DataFrame(columns=columns)

decision_maker = DecisionMaker(32, 32)
decision_maker.load_state_dict(torch.load(BASE_PATH + 'decision_maker/32_model.pt'))

id2team = {v: k for k, v in team2id.items()}

decision_maker.eval()

with torch.inference_mode():
    i = 0
    for x, y in zip(X_test, y_test):
        match, matches_ab, matches_a, matches_b, _, bets= x
        match = torch.unsqueeze(match, 0)
        matches_ab = torch.unsqueeze(matches_ab, 0)
        matches_a = torch.unsqueeze(matches_a, 0)
        matches_b = torch.unsqueeze(matches_b, 0)
        bets = torch.unsqueeze(torch.tensor(bets, dtype=torch.float32), 0)

        pred = best_acc_model(match, matches_ab, matches_a, matches_b)
        match_between = match[:, :, :1].type(torch.int)
        how_much_to_invest = decision_maker(match_between, pred, bets)

        best_acc_logs.loc[i] = [y.item(),
                                torch.argmax(pred).item(),
                                torch.max(pred).item(),
                                *pred.tolist()[0],
                                id2team[match[0, 0, 0].type(torch.int).item()],
                                id2team[match[0, 1, 0].type(torch.int).item()],
                                how_much_to_invest.item(),
                                 *bets.tolist()[0]]

        pred = best_loss_model(match, matches_ab, matches_a, matches_b)
        best_loss_logs.loc[i] = [y.item(),
                                torch.argmax(pred).item(),
                                torch.max(pred).item(),
                                *pred.tolist()[0],
                                id2team[match[0, 0, 0].type(torch.int).item()],
                                id2team[match[0, 1, 0].type(torch.int).item()],
                                how_much_to_invest.item(),
                                 *bets.tolist()[0]]

        i+=1

best_acc_logs.to_csv(BASE_PATH + "best_acc_logs.csv", index=False)
best_loss_logs.to_csv(BASE_PATH + "best_loss_logs.csv", index=False)